In [1]:
import pydub
import ipywidgets as widgets
from IPython.display import Audio, display #pour écouter les extraits audio avec un notebook jupyter



In [2]:
import time
import csv
import pandas as pd
import datetime
import os.path

In [3]:
path_itw = "../records/grazia_borrini_07-06-18_part-1.MP3"

original_trancript_path = "../records/grazia_borrini_07-06-18.csv"
cleaned_trancript_path = "../records/2026-02-02_itw_borrini_cleaned.csv"

In [4]:
itw = pydub.AudioSegment.from_mp3(path_itw) 


In [5]:

csvfile = open(original_trancript_path, newline='')
csv_reader = csv.DictReader(csvfile)
rows = list(csv_reader)



def check_uncleaned_segment():
    uncleaned_row = []
    if os.path.exists(cleaned_trancript_path):
            with open(cleaned_trancript_path, newline = '') as csv2:
                reader2 = csv.DictReader(csv2)
                cleaned_rows = [rows["id"] for rows in reader2]
            with open(original_trancript_path, newline = '') as csv1:
                reader = csv.DictReader(csv1)
                uncleaned_row = [row for row in reader if row['id'] not in cleaned_rows]
    else:
        with open(original_trancript_path, newline = '') as csv1:
                reader = csv.DictReader(csv1)
                uncleaned_row = [row for row in reader]
    return uncleaned_row

                    
    


print(len(check_uncleaned_segment()), len(rows))


934 934


In [ ]:
caption = widgets.Label(value='The slider value is in its initial position.')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    ) + str(change.new)

slider.observe(handle_slider_change, names='value')

display(caption, slider)

In [22]:
button = widgets.Button(description="Next")
caption = widgets.Label(value= None)
caption2 = widgets.Label(value= None)
        
output = widgets.Output()

i = 0

def on_button_clicked2(bb, ):
   
    with output:
        uncleaned_row = check_uncleaned_segment()
        print("new len of uncleaned_row : ", len(uncleaned_row))
        old_speaker_name = uncleaned_row[i]['speaker']
        old_text = uncleaned_row[i]['text']
        if old_speaker_name != caption.value:
            print("Le nom du speaker a été remplacé par : ", caption.value)
        else :
            print("Le nom du speaker n'a pas été modifié")
        if old_text != caption2.value:
            print("Le texte a été modifié : ", caption2.value)
        else:
            print("Le texte n'a pas été modifié!")
        
        
        if os.path.exists(cleaned_trancript_path):
            with open(cleaned_trancript_path, 'a', newline='') as csvfile:
                fieldnames = ['id','start', 'end','speaker','cleaned_speaker', 'text','cleaned_text', 'start_in_ms','end_in_ms']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'id': uncleaned_row[i]['id'],
                                 'start': uncleaned_row[i]["start"],
                                 'end': uncleaned_row[i]["end"],
                                 'speaker': uncleaned_row[i]['speaker'],
                                 'cleaned_speaker':caption.value,
                                 'text':uncleaned_row[i]['text'].strip(),
                                 'cleaned_text' : caption2.value,
                                 'start_in_ms':uncleaned_row[i]['start_in_ms'],
                                 'end_in_ms':uncleaned_row[i]['end_in_ms']})
        else:
            with open(cleaned_trancript_path , 'w', newline='') as csvfile:
                fieldnames = ['id','start', 'end','speaker', 'cleaned_speaker', 'text','cleaned_text', 'start_in_ms','end_in_ms']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerow({'id': uncleaned_row[i]['id'],
                                 'start': uncleaned_row[i]["start"], 
                                 'end': uncleaned_row[i]["end"], 
                                 'speaker': uncleaned_row[i]['speaker'],
                                 'cleaned_speaker':caption.value, 
                                 'text':uncleaned_row[i]['text'].strip(), 
                                 'cleaned_text' : caption2.value, 
                                 'start_in_ms':uncleaned_row[i]['start_in_ms'], 
                                 'end_in_ms':uncleaned_row[i]['end_in_ms']})

            

def on_button_clicked(b):
    global i
    with output:
        output.clear_output()
        uncleaned_row = check_uncleaned_segment()
        print("len of uncleaned_row : ", len(uncleaned_row))
        extract = itw[float(uncleaned_row[i]['start_in_ms'])*1000:float(uncleaned_row[i]['end_in_ms'])*1000]
        display(extract)
        new_speaker = widgets.Text(value = uncleaned_row[i]['speaker'].strip(),
                        placeholder='Speaker name',
                        description='Speaker:',
                        disabled=False   
                    )
        #caption = widgets.Label(value='LA')
        new_text = widgets.Textarea(
            value=uncleaned_row[i]['text'].strip(),
            placeholder='Type something',
            description='String:',
            disabled=False,
            layout=widgets.Layout(width='50%', height='80px')
            )
        display(new_speaker)
        l = widgets.jslink((new_speaker, 'value'), (caption, 'value'))
        display(new_text)
        caption.value = str(new_speaker.value)
        caption2.value = str(new_text.value)
        button2 = widgets.Button(description="Save")
        new_speaker.observe(handle_speaker_change, names='value')
        new_text.observe(handle_text_change, names='value')

        display(button2)
        button2.on_click(on_button_clicked2)
        old_speaker_name = uncleaned_row[i]['speaker']

        #print("Speaker value : ", caption.value)
        #print("Text value : ", caption2.value)
        #i+=1
        


        

def handle_speaker_change(change):
    caption.value = str(change.new)

    
def handle_text_change(change):
    caption2.value = str(change.new)




display(output,button)

button.on_click(on_button_clicked)


Output()

Button(description='Next', style=ButtonStyle())

In [23]:
button = widgets.Button(description="Next")
caption = widgets.Label(value= None)
caption2 = widgets.Label(value= None)
        
output = widgets.Output()

i = 0

def save_change(row_to_clean, speaker, text):
        if os.path.exists(cleaned_trancript_path):
            with open(cleaned_trancript_path, 'a', newline='') as csvfile:
                fieldnames = ['id','start', 'end','speaker','cleaned_speaker', 'text','cleaned_text', 'start_in_ms','end_in_ms']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'id': row_to_clean['id'],
                                 'start': row_to_clean["start"],
                                 'end': row_to_clean["end"],
                                 'speaker': row_to_clean['speaker'],
                                 'cleaned_speaker':caption.value,
                                 'text':row_to_clean['text'].strip(),
                                 'cleaned_text' : caption2.value,
                                 'start_in_ms':row_to_clean['start_in_ms'],
                                 'end_in_ms':row_to_clean['end_in_ms']})
        else:
            with open(cleaned_trancript_path , 'w', newline='') as csvfile:
                fieldnames = ['id','start', 'end','speaker', 'cleaned_speaker', 'text','cleaned_text', 'start_in_ms','end_in_ms']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerow({'id': row_to_clean['id'],
                                 'start': row_to_clean["start"], 
                                 'end': row_to_clean["end"], 
                                 'speaker': row_to_clean['speaker'],
                                 'cleaned_speaker':caption.value, 
                                 'text':row_to_clean['text'].strip(), 
                                 'cleaned_text' : caption2.value, 
                                 'start_in_ms':row_to_clean['start_in_ms'], 
                                 'end_in_ms':row_to_clean['end_in_ms']})

            

def on_button_clicked(b):
    global i
    with output:
        print(caption.value, caption2.value)
        uncleaned_row = check_uncleaned_segment()
        save_change(uncleaned_row[0], speaker = caption.value, text=caption2.value)
        output.clear_output()
        
        print("len of uncleaned_row : ", len(uncleaned_row))
        extract = itw[float(uncleaned_row[i]['start_in_ms'])*1000:float(uncleaned_row[i]['end_in_ms'])*1000]
        display(extract)
        new_speaker = widgets.Text(value = uncleaned_row[i]['speaker'].strip(),
                        placeholder='Speaker name',
                        description='Speaker:',
                        disabled=False   
                    )
        #caption = widgets.Label(value='LA')
        new_text = widgets.Textarea(
            value=uncleaned_row[i]['text'].strip(),
            placeholder='Type something',
            description='String:',
            disabled=False,
            layout=widgets.Layout(width='50%', height='80px')
            )
        display(new_speaker)
        l = widgets.jslink((new_speaker, 'value'), (caption, 'value'))
        display(new_text)
        caption.value = str(new_speaker.value)
        caption2.value = str(new_text.value)
        button2 = widgets.Button(description="Save")
        new_speaker.observe(handle_speaker_change, names='value')
        new_text.observe(handle_text_change, names='value')

        #display(button2)
        #button2.on_click(on_button_clicked2)
        old_speaker_name = uncleaned_row[i]['speaker']

        #print("Speaker value : ", caption.value)
        #print("Text value : ", caption2.value)
        #i+=1
        


        

def handle_speaker_change(change):
    caption.value = str(change.new)

    
def handle_text_change(change):
    caption2.value = str(change.new)




display(output,button)

button.on_click(on_button_clicked)


Output()

Button(description='Next', style=ButtonStyle())